In [1]:
import torchvision.models as torchmodels
from torch import nn
import utils as u

from filterbank import STFT, MelFilter, Log1p, MedFilt
import torch
import torch.nn.functional as F
from transformers import AutoProcessor, ClapModel, ClapAudioModelWithProjection, ClapProcessor
import torch.nn.functional as F
torch.hub.set_dir('/data/woutdecrop/torch/')

import datetime
import json
import os
import pathlib
import shutil
import sys

# import fairseq
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import scipy
import soundfile as sf
import torch
import torchaudio
import torchaudio.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from PIL import Image
# from maad import util
from tqdm import tqdm
import torch.optim as optim
from sklearn.model_selection import train_test_split
import copy
import suntime
import pytz

from transformers import ClapModel, ClapProcessor
from transformers import pipeline

import models
import utils as u

torchaudio.set_audio_backend(backend='soundfile')

# matplotlib.use('TkAgg')
# Get the color map by name:
cm = plt.get_cmap('jet')

from dataset import LifeWatchDataset
import yaml
import pathlib
import numpy as np
config_path=r"config.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)



/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_10875/2082739506.py:46: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend(backend='soundfile')
/srv/CLAP/dataset.py:36: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend(backend='soundfile')


In [1]:
import os

# Example directory path
result_dir = '/srv/CLAP/3_1_Results/2024-08-08_12-30_a5_b0.5_weights_0.1_0.2_0.3_0.4_'

# Extract folder name
folder_name = os.path.basename(result_dir.rstrip('/'))
print(folder_name)


2024-08-08_12-30_a5_b0.5_weights_0.1_0.2_0.3_0.4_


In [8]:
batch_size=8
stop_shuffle=False
sample_dur=10
desired_fs = 48000
wavs_folder= '/storage/Imagine_UC6_new/DATA/data_per_station_4_updated_metadata_extra_filter-window-4'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Selected CUDA device:", torch.cuda.get_device_name(device))
else:
    print("CUDA is not available. Using CPU.")
# device =torch.device('cuda:0')
# log_path = 'roi/BioLingual/logs_test.log'
# log_file = open(log_path, mode='w')

# detections = convert_raven_to_ae_format(labels_to_exclude=None)
# detections = detections.loc[~detections.label.isna()]

d_train_path= "ds_split-CLAP/train.txt"
d_test_path= "ds_split-CLAP/test.txt"
# print("path ", d_train_path)
split = np.genfromtxt(d_train_path, dtype='str', delimiter=' ')
d_train_loc = np.array([os.path.join(wavs_folder, i) for i in split[:, 0]])

split = np.genfromtxt(d_test_path, dtype='str', delimiter=' ')
d_test_loc = np.array([os.path.join(wavs_folder, i) for i in split[:, 0]])

# split = np.genfromtxt(d_train_path, dtype='str', delimiter=' ')
# d_train_loc = np.array([os.path.join(wavs_folder, i) for i in split[:, 0]])

# split = np.genfromtxt(d_valid_path, dtype='str', delimiter=' ')
# d_valid_loc = np.array([os.path.join(wavs_folder, i) for i in split[:, 0]])
d_train=u.process_filenames(d_train_loc)
d_test=u.process_filenames(d_test_loc)

train_labels = set(d_train["label"])
# Count the occurrences of each label in d_train
label_counts_train = d_train["label"].value_counts()

valid_train_labels = label_counts_train[label_counts_train >= 5].index


# # Filter d_train and d_valid based on valid_train_labels
# d_train = d_train[d_train["label"].isin(valid_train_labels)]
# test_train_labels = label_counts_train[label_counts_train >= 5].index

# d_test = d_test[d_test["label"].isin(test_train_labels)]

# # Display the number of unique classes
# num_classes = len(valid_train_labels)
# ids={lbl: i for i, lbl in enumerate(d_train['label'].unique())}

# similarity_matrix=u.similarity(ids,device,distance_weight = 0.6, speed_weight = 0.30,activity_weight = 0.10)
# similarity_matrix_distance=u.similarity(ids,device,distance_weight = 1)
# similarity_matrix_speed=u.similarity(ids,device,speed_weight = 1)
# similarity_matrix_activity=u.similarity(ids,device,activity_weight = 1)




Selected CUDA device: Tesla T4


In [9]:
valid_train_labels

Index(['Cargo at distance 4-5 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 3-4 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 1-2 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 5-6 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 2-3 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 3-4 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 4-5 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 2-3 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 5-6 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 6-7 km with speed 0-1.5 is at-anchor',
       'Cargo at distance 1-2 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 6-7 km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 10+ km with speed 8.2-13.3 is underway-using-engine',
       'Cargo at distance 0-1 km with speed 0-1.5 

In [19]:

# from msclap import CLAP

# Load and initialize CLAP
model_path='davidrrobinson/BioLingual'
clap_model=ClapAudioModelWithProjection.from_pretrained(model_path)
clap_model = ClapModel.from_pretrained(pretrained_model_name_or_path = 'davidrrobinson/BioLingual')

#Load audio files
# audio_files = ['audio_file']

# Generate captions for the recording
captions = clap_model.generate_caption(audio_files, resample=True, beam_size=5, entry_length=67, temperature=0.01)

# Print the result
for i in range(len(audio_files)):
    print(f"Audio file: {audio_files[i]} \n")
    print(f"Generated caption: {captions[i]} \n")



AttributeError: 'ClapModel' object has no attribute 'generate_caption'

In [28]:
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
model.generate_caption(audio_files)

AttributeError: 'ClapModel' object has no attribute 'generate_caption'

In [4]:
audio_files=audio_files[1:3]

In [5]:
from msclap import CLAP

# Load model (Choose version 'clapcap')
clap_model = CLAP(version = 'clapcap', use_cuda=False)
captions = clap_model.generate_caption(audio_files)

In [7]:
for i in range(len(audio_files)):
    print(f"Audio file: {audio_files.iloc[i]} \n")
    print(f"Generated caption: {captions[i]} \n")


Audio file: /storage/UC6_new_data/data/Grafton_15810_train/Grafton_15810_2022-05-08_18-55-33_267-0_Cargo_at-anchor_0-0_2022-05-08-19-00-00_2169.wav 

Generated caption: A machine is running at a steady pace.  

Audio file: /storage/UC6_new_data/data/Grafton_15810_train/Grafton_15810_2022-03-01_20-37-50_130-0_Cargo_at-anchor_0-0_2022-03-01-20-40-00_5666.wav 

Generated caption: The rain is falling and the wind is blowing.  

